# Imports

In [147]:
import os
import pandas as pd
import numpy as np
import scipy
import pickle
import sys

## Helper Functions

In [148]:
%store -r loadData_hf
sys.path.insert(0, loadData_hf)
import LoadNetworkDataByID as lndbi

# Set Directory Paths

In [149]:
# Location of the data folder
%store -r dataDir

# Directory path where Data will be saved to
%store -r path_dataDir

# Directory path where Data will be saved to
%store -r thick_dataDir

# [1] Loading Raw Thickness Data

In [150]:
# MRI Thickness value for All Subjects - schaefer400x7
thicknessAllraw = pd.read_csv(os.path.join(dataDir, 'NewFTDData', 
                                           'invivoPathCohort_quantsSubSesSchaefer400_tian12.csv'), dtype={'id': str})

### Loading Look Up Table for Type of MRI Thickness Subjects

In [151]:
thicknessPathLUT = pd.read_excel(os.path.join(dataDir, 'NewFTDData', 'InvivoPathCohort_03172023.xls'), 
                                 dtype={'INDDID': str})

### Join the Above two dataframes on INDDID (Keep only the ones that INDDID are overlapping)

In [152]:
thicknessAll = pd.merge(thicknessAllraw, thicknessPathLUT, left_on='id', right_on='INDDID', how='inner') 

# We only lose INDDID 108783x09 in the thicknessAllraw (849 rows lost)

### Group by path type

In [153]:
thickness_path_type = thicknessAll.groupby('Group')

# MRI Thickness values for Healthy Control
thicknessHC = thickness_path_type.get_group('HC')
# MRI Thickness values for Patient (TAU)
thicknessPatientTAU = thickness_path_type.get_group('tau')
# MRI Thickness values for Patient (TDP)
thicknessPatientTDP = thickness_path_type.get_group('tdp')

In [154]:
# IDs
HC_IDs = np.unique(thicknessHC.INDDID)
TAU_IDs = np.unique(thicknessPatientTAU.INDDID)
TDP_IDs = np.unique(thicknessPatientTDP.INDDID)

# Get only that Matches Pathology

In [128]:
thick_id_TAU = TAU_IDs.astype(np.float64)
thick_id_TDP = TDP_IDs.astype(np.float64)

In [131]:
# Loading path_id_TAU
with open(os.path.join(path_dataDir, 'path_id_TAU.pkl'), 'rb') as f:
    path_id_TAU = pickle.load(f)
f.close()
    
# Loading path_id_TDP
with open(os.path.join(path_dataDir, 'path_id_TDP.pkl'), 'rb') as f:
    path_id_TDP = pickle.load(f)
f.close()

In [132]:
mr_path_tau = np.isin(thick_id_TAU, path_id_TAU)
print(f"Number of overlap (Tau - Path vs MR): {mr_path_tau.sum()}/{len(thick_id_TAU)}")

Number of overlap (Tau - Path vs MR): 13/26


In [133]:
mr_path_tdp = np.isin(thick_id_TDP, path_id_TDP)
print(f"Number of overlap (TDP - Path vs MR): {mr_path_tdp.sum()}/{len(thick_id_TDP)}")

Number of overlap (TDP - Path vs MR): 21/30


In [175]:
mr_path_tau_id = TAU_IDs[mr_path_tau]
mr_path_tdp_id = TDP_IDs[mr_path_tdp]

In [176]:
mr_path_tdp_id.dtype

dtype('O')

## TAU

In [177]:
thicknessPatientTAU_Path = thicknessPatientTAU[thicknessPatientTAU['INDDID'].isin(mr_path_tau_id)]

## TDP

In [178]:
thicknessPatientTDP_Path = thicknessPatientTDP[thicknessPatientTDP['INDDID'].isin(mr_path_tdp_id)]

# [2] Formatting Thickness Data

### numLab (Number of Label in Schaefer400x7 Atlas)

In [183]:
numLab=400

### Thickness data for HC, TAU, TDP

In [184]:
# Get thickness mean and volume total values for [Control MRI data IDs (54) x lables (numLab)] 26 x 400
HCResults = lndbi.LoadNetworkDataByID(HC_IDs, thicknessHC,'Schaefer400x7v1', numLab = 400)

# Get thickness mean and volume total values for 
# [Patient (TAU) MRI data IDs (26) x lables (numLab = 400 regions in the sch region)] / 26 x 400
PatientTAUResults = lndbi.LoadNetworkDataByID(TAU_IDs, thicknessPatientTAU,'Schaefer400x7v1', 
                                        numLab = 400)

# Get thickness mean and volume total values for 
# [Patient (TDP) MRI data IDs (30) x lables (numLab = 400 regions in the sch region)] / 26 x 400
PatientTDPResults = lndbi.LoadNetworkDataByID(TDP_IDs, thicknessPatientTDP,'Schaefer400x7v1', 
                                        numLab = 400)

1
100115
2
100338


/Users/hyroh/Desktop/FTD_Research/mBIN/FTD_JupyterNotebook/HelperFunctions/Load_Dataset_HelperFunctions/LoadNetworkDataByID.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  AllResults['Thickness']['Mean'][i, L] = currThickness[currLabelsThick == L+1]
/Users/hyroh/Desktop/FTD_Research/mBIN/FTD_JupyterNotebook/HelperFunctions/Load_Dataset_HelperFunctions/LoadNetworkDataByID.py:115: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  AllResults['Volume']['Total'][i, L] = currVolume[currLabelsVol == L+1]


3
100387
4
100488
5
100742
6
101080
7
102187
8
102429
9
102967
10
102974
11
103023
12
103073
13
103118
14
103824
15
104102
16
105521
17
105601
18
106036
19
106135
20
106732
21
106897
22
107045
23
107700
24
108014
25
108560
26
108853
27
108856
28
109075
29
109194
30
109212
31
109325
32
109346
33
109417
34
109481
35
110342
36
111151
37
111611
38
112359
39
112916
40
113326
41
115226
42
115311
43
115629
44
116543
45
117957
46
117964
47
118177
48
118752
49
118885
50
119158
51
119535
52
119949
53
120506
54
124747
1
101483
2
104093
3
104190
4
105564
5
106309
6
106814
7
107516
8
107677
9
108026
10
109300
11
113113
12
115001
13
116591
14
116607
15
118064
16
118148
17
118410
18
118780
19
119089
20
120594
21
122104
22
122375
23
122376
24
122419
25
122601
26
123352
1
101045
2
104094
3
104862
4
105223
5
108276
6
108344
7
108790
8
109050
9
109073
10
109476
11
112273
12
112780
13
112974
14
114753
15
115264
16
117589
17
117637
18
118234
19
118430
20
118914
21
119349
22
119454
23
119502
24
120141
25
12

## Matching Path INDDIDs

In [185]:
PatientTAUResults_Path = lndbi.LoadNetworkDataByID(mr_path_tau_id, thicknessPatientTAU_Path,'Schaefer400x7v1', 
                                        numLab = 400)
PatientTDPResults_Path = lndbi.LoadNetworkDataByID(mr_path_tdp_id, thicknessPatientTDP_Path,'Schaefer400x7v1', 
                                        numLab = 400)

1
101483
2
105564
3
106309
4
106814
5
107516
6
107677
7
108026
8
113113
9
115001
10
116591
11
116607
12
118410
13
118780
1
101045
2
104094
3
104862
4
108276
5
108344
6
108790
7
109050
8
109073
9
109476
10
112273
11
112780
12
112974
13
114753
14
117589
15
117637
16
118234
17
118430
18
118914
19
119454
20
120950
21
121199


# Save the Dataset and Variables

#### Save HCResults, PatientTAUResults, and PatientTDPResults

In [188]:
# Save HCResults
with open(os.path.join(thick_dataDir, 'HCResults.pkl'), 'wb') as f:
    pickle.dump(HCResults, f)
f.close()

# Save PatientTAUResults
with open(os.path.join(thick_dataDir, 'PatientTAUResults.pkl'), 'wb') as f:
    pickle.dump(PatientTAUResults, f)
f.close()

# Save PatientTDPResults
with open(os.path.join(thick_dataDir, 'PatientTDPResults.pkl'), 'wb') as f:
    pickle.dump(PatientTDPResults, f)
f.close()

# Save PatientTAUResults_Path
with open(os.path.join(thick_dataDir, 'PatientTAUResults_Path.pkl'), 'wb') as f:
    pickle.dump(PatientTAUResults_Path, f)
f.close()

# Save PatientTDPResults_Path
with open(os.path.join(thick_dataDir, 'PatientTDPResults_Path.pkl'), 'wb') as f:
    pickle.dump(PatientTDPResults_Path, f)
f.close()

# Data Summary

In [190]:
print("Length of HC IDs in Thickness: " + str(len(HC_IDs)))
print("Length of TAU IDs in Thickness: " + str(len(TAU_IDs)))
print("Length of TDP IDs in Thickness: " + str(len(TDP_IDs)))

print("Length of TAU IDs in Thickness (Matching path INDDID): " + str(len(mr_path_tau_id)))
print("Length of TDP IDs in Thickness (Matching path INDDID): " + str(len(mr_path_tdp_id)))

Length of HC IDs in Thickness: 54
Length of TAU IDs in Thickness: 26
Length of TDP IDs in Thickness: 30
Length of TAU IDs in Thickness (Matching path INDDID): 13
Length of TDP IDs in Thickness (Matching path INDDID): 21
